In [1]:
import pandas as pd

# import stravalib
import flask
from flask import request
from stravalib import Client
import requests
import json
import webbrowser
import pandas as pd
import datetime

from bokeh.plotting import figure, output_file, show, output_notebook, ColumnDataSource
from bokeh.palettes import Spectral11
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, HoverTool
from bokeh.layouts import gridplot
from bokeh.core.properties import value
# from ... import strava_creds

import warnings
warnings.simplefilter('ignore')

In [2]:
with open('../../strava_creds/strava_creds.json') as scr:
    strava_creds = json.load(scr)

In [3]:
strava_creds

{'ACCESS_TOKEN': '481cb676c1ebd336da0c25230f837ce8ac6190ad',
 'CODE': '3c66e15a369292748e3ab195b0162487b983c6ae',
 'CLIENT_ID': 24706,
 'CODE_MAC': '97a0338d3555c9640789434e93e2f52da57300de'}

In [4]:
client = Client()
# authorize_url = client.authorization_url(client_id=24706, redirect_uri='http://localhost:8282/authorized')

# url = client.authorization_url(client_id=24706,redirect_uri='http://127.0.0.1:5000/authorization')
client_id = strava_creds['CLIENT_ID']

access_token = strava_creds['ACCESS_TOKEN']
port = 5000
url = 'http://localhost:%d/authorized' % port
authorize_url = client.authorization_url(client_id=client_id, redirect_uri=url)

webbrowser.open(authorize_url)

code = strava_creds['CODE_MAC']

access_token = client.exchange_code_for_token(client_id=client_id, client_secret=access_token, code=code)


In [5]:
# activities_returned = []
# activity_returned = dict()

In [6]:
activities_list = [(activity.start_date,activity.name,activity.distance) for activity in client.get_activities(after = "2018-01-01T00:00:00Z",  limit=800)]


In [7]:
extract_df = pd.DataFrame.from_dict(activities_list)
len(extract_df)

212

In [8]:
extract_df = extract_df.rename(columns={0: 'run_date',
                          1: 'run_title',
                          2: 'run_distance_m'})

In [9]:
extract_df['kms'] = extract_df['run_distance_m']/1000

In [10]:
extract_df.index = extract_df['run_date']

In [11]:
extract_df = extract_df[['run_title', 'run_distance_m', 'kms']]

In [12]:
daily_runs_series = extract_df.resample('D')['kms'].sum()

In [13]:
daily_runs_df = pd.DataFrame(daily_runs_series)

In [14]:
def to_float(number):
    return float(number)

In [15]:
def convert_week_number(week_number):
    return f'week {week_number}'

In [16]:
def cumulative_per_week(df, week_number):
    df = df.copy(deep=True)
    df_week = df[df['week_number']==week_number]
    
    df_week['weekly_actual_total'] = df_week['kms'].sum()
    df_week['weekly_actual_cumulative'] = df_week['kms'].cumsum()
    df_week['percentage_to_total_weekly'] = df_week['weekly_actual_cumulative']/df_week['weekly_actual_total']
    df_week['weekly_goal_total'] = df_week['daily_goal'].sum()
    df_week['weekly_goal_cumulative'] = df_week['daily_goal'].cumsum()
    df_week['percentage_to_total_weekly_goal'] = df_week['weekly_goal_cumulative']/df_week['weekly_goal_total']
#     df_week['weekly_goal_left'] = df_week['weekly_goal_total'] - df_week['weekly_actual_cumulative']
    
    day_number_range = range(1, len(df_week)+1)
    if len(df_week) == 7:
        day_of_week = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
        df_week['day_of_week'] = day_of_week
    else:
        df_week['day_of_week'] = None
        
    df_week['day_number'] = day_number_range
    
    return df_week

In [17]:
daily_runs_df['kms'] = daily_runs_df.apply(lambda row: to_float(row['kms']), axis=1)

In [18]:
daily_runs_df['cumulative_daily_actual_kms'] = daily_runs_df['kms'].cumsum()
daily_runs_df['daily_goal'] = 20/7
daily_runs_df['cumulative_daily_goal'] = daily_runs_df['daily_goal'].cumsum()
# daily_runs_df['daily_km_difference'] = 

In [19]:
daily_runs_df['daily_kms_difference'] = daily_runs_df['kms']/daily_runs_df['daily_goal']
daily_runs_df['daily_kms_cumulative_difference'] = daily_runs_df['cumulative_daily_actual_kms']/daily_runs_df['cumulative_daily_goal']

In [20]:
daily_runs_df.to_csv('strava_kms_runs_2018.csv')

In [21]:
weekly_kms_df = daily_runs_df.resample('W-MON').sum()

In [22]:
weekly_kms_df = weekly_kms_df[['kms', 'daily_goal']]

In [23]:
weekly_kms_df = weekly_kms_df.rename(columns={'daily_goal': 'weekly_goal'})

In [24]:
weekly_kms_df['cumulative_weekly_kms'] = weekly_kms_df['kms'].cumsum()
weekly_kms_df['cumulative_weekly_goal_kms'] = weekly_kms_df['weekly_goal'].cumsum()

In [25]:
weekly_kms_df['week_number'] = weekly_kms_df.index.week
daily_runs_df['week_number'] = daily_runs_df.index.week

In [26]:
weekly_kms_df['week'] = weekly_kms_df.apply(lambda row: convert_week_number(row['week_number']), axis=1)
daily_runs_df['week'] = daily_runs_df.apply(lambda row: convert_week_number(row['week_number']), axis=1)

In [27]:
weekly_kms_df = weekly_kms_df.round(2)
daily_runs_df = daily_runs_df.round(2)

In [28]:
weekly_num = list(set(list(daily_runs_df['week_number'])))

In [29]:
daily_weekly_kms_df = pd.DataFrame()

In [30]:
for week in weekly_num:
    transition_df = cumulative_per_week(daily_runs_df, week)
    daily_weekly_kms_df = daily_weekly_kms_df.append(transition_df)
    

In [31]:
daily_weekly_kms_df['daily_kms_cumulative_difference'] = daily_weekly_kms_df['cumulative_daily_actual_kms'] - daily_weekly_kms_df['weekly_goal_total']

In [54]:
daily_weekly_kms_df.head()

,kms,cumulative_daily_actual_kms,daily_goal,cumulative_daily_goal,daily_kms_difference,daily_kms_cumulative_difference,week_number,week,weekly_actual_total,weekly_actual_cumulative,percentage_to_total_weekly,weekly_goal_total,weekly_goal_cumulative,percentage_to_total_weekly_goal,day_of_week,day_number,total_kms,diff_actual_vs_total,daily_kms_cumulative_difference_abs,annual_total
run_date,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00+00:00,5.57,5.57,2.86,2.86,1.95,-34.47,1,week 1.0,62.89,5.57,0.088567,40.04,2.86,0.071429,None,1,1056.49,57.32,34.47,1056.49
2018-01-02 00:00:00+00:00,5.77,11.34,2.86,5.71,2.02,-28.70,1,week 1.0,62.89,11.34,0.180315,40.04,5.72,0.142857,None,2,1056.49,51.55,28.70,1056.49
2018-01-03 00:00:00+00:00,4.95,16.29,2.86,8.57,1.73,-23.75,1,week 1.0,62.89,16.29,0.259024,40.04,8.58,0.214286,None,3,1056.49,46.60,23.75,1056.49
2018-01-04 00:00:00+00:00,0.75,17.04,2.86,11.43,0.26,-23.00,1,week 1.0,62.89,17.04,0.270949,40.04,11.44,0.285714,None,4,1056.49,45.85,23.00,1056.49
2018-01-05 00:00:00+00:00,5.65,22.69,2.86,14.29,1.98,-17.35,1,week 1.0,62.89,22.69,0.360789,40.04,14.30,0.357143,None,5,1056.49,40.20,17.35,1056.49


In [33]:
daily_weekly_kms_df['day_number'] = list(range(1, len(daily_weekly_kms_df)+1))
daily_weekly_kms_df['total_kms'] = daily_weekly_kms_df['kms'].sum()

In [34]:
daily_weekly_kms_df['diff_actual_vs_total'] = daily_weekly_kms_df['weekly_actual_total'] - daily_weekly_kms_df['weekly_actual_cumulative']

In [35]:
daily_weekly_kms_df.to_csv('strava_data.csv')

In [36]:
output_file('grid_plot.html')

In [37]:
# output_file('strava_chart3.html')

source = ColumnDataSource(data=weekly_kms_df)

km_fig = figure(
                  plot_height=300, plot_width=600,
                  title='cumulative actual distance vs goal actual distance',
                  x_axis_label='week', y_axis_label='Kms',
                  toolbar_location=None)


km_fig.line(x='week_number', y='cumulative_weekly_kms', 
         color='blue', line_width=1,
         legend='Cumulative weekly kms',source=source)

km_fig.vbar(x='week_number', bottom=0, top='cumulative_weekly_kms', 
         color='blue', width=0.75, 
         legend='Actual', source=source)


km_fig.line(x='week_number', y='cumulative_weekly_goal_kms', 
         color='red', line_width=1,
         legend='Cumulative goal kms',source=source)

km_fig.legend.location = 'top_left'

# show(km_fig)

In [38]:
x_axis = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']

In [39]:
day_week_cds = ColumnDataSource(daily_weekly_kms_df)


km_week_comp_fig = figure(plot_height=300, plot_width=600,
                  title='cumulative actual distance vs goal actual distance',
                  x_axis_label='Day of the week 1.0', y_axis_label='Kms',
                  x_range=x_axis,
                  toolbar_location=None, y_range=(0, 50))

week_1_view = CDSView(source=day_week_cds,
                       filters=[GroupFilter(column_name='week', group='week 1.0')])
week_2_view = CDSView(source=day_week_cds,
                        filters=[GroupFilter(column_name='week', group='week 1.0')])

km_week_comp_fig.line(x='day_of_week', y='cumulative_daily_actual_kms', 
         color='blue', line_width=1,
         legend='Cumulative',  source=day_week_cds, view=week_1_view)

km_week_comp_fig.line(x='day_of_week', y='cumulative_daily_goal', 
         color='red', line_width=1,
         legend='Cumulative goal', source=day_week_cds, view=week_2_view)

# Move the legend to the upper left corner
km_week_comp_fig.legend.location = 'top_left'

# Show the plot
# show(km_week_comp_fig)

In [40]:
weekly_kms_df.head()

,kms,weekly_goal,cumulative_weekly_kms,cumulative_weekly_goal_kms,week_number,week
run_date,,,,,,
2018-01-01 00:00:00+00:00,5.57,2.86,5.57,2.86,1,week 1.0
2018-01-08 00:00:00+00:00,30.55,20.00,36.12,22.86,2,week 2.0
2018-01-15 00:00:00+00:00,38.93,20.00,75.05,42.86,3,week 3.0
2018-01-22 00:00:00+00:00,35.69,20.00,110.74,62.86,4,week 4.0
2018-01-29 00:00:00+00:00,21.71,20.00,132.45,82.86,5,week 5.0


In [41]:
# output_file('strava_chart3.html')

cds_bar = ColumnDataSource(data=weekly_kms_df)

X_AXIS_SUMMARY = list(weekly_kms_df['week'].unique())

# Create a figure with a datetime type x-axis
fig = figure(title='actual versus goals',
             plot_height=300, plot_width=600,
             x_axis_label='Week number', y_axis_label='KMs',
             x_minor_ticks=2, y_range=(0, 50),
             x_range=X_AXIS_SUMMARY,
             toolbar_location=None)

# The daily words will be represented as vertical bars (columns)
fig.vbar(x='week', bottom=0, top='kms', 
         color='blue', width=0.75, 
         legend='Actual', source=cds_bar)

# The cumulative sum will be a trend line
fig.line(x='week', y='weekly_goal', 
         color='gray', line_width=1,
         legend='Goal', source=cds_bar)

# Put the legend in the upper left corner
fig.legend.location = 'top_left'

tooltips = [
            ('Kilometers','@kms'),
            ('Week number', '@week'),
           ]

# Add the HoverTool to the figure
fig.add_tools(HoverTool(tooltips=tooltips))


### Weekly bar line to show atual vs goal kms


In [42]:
daily_weekly_kms_df['daily_kms_cumulative_difference_abs'] = daily_weekly_kms_df['daily_kms_cumulative_difference'].abs()

In [43]:
df4 = daily_weekly_kms_df[daily_weekly_kms_df['week']=='week 2.0']


In [44]:
df4 = df4[['daily_kms_cumulative_difference_abs',
              'weekly_actual_cumulative', 'day_of_week']]

In [45]:
# output_file("stacked.html")

source = ColumnDataSource(df4)

# week_2_view = CDSView(source=source,
#                         filters=[GroupFilter(column_name='week', group='week 2.0')])

stack_kms_goals = figure(x_range=x_axis, plot_height=300, plot_width=600, title="runs by day",
           toolbar_location=None, tools="")

colors = ['red', 'blue']
km_type = ['daily_kms_cumulative_difference_abs', 'weekly_actual_cumulative']

stack_kms_goals.vbar_stack(km_type, x='day_of_week', width=0.9, color=colors, 
              source=source)
#               view=week_2_view)

stack_kms_goals.y_range.start = 0
stack_kms_goals.x_range.range_padding = 0.1
stack_kms_goals.xgrid.grid_line_color = None
stack_kms_goals.axis.minor_tick_line_color = None
stack_kms_goals.outline_line_color = None
stack_kms_goals.legend.location = "top_left"
stack_kms_goals.legend.orientation = "horizontal"
# show(stack_kms_goals)

In [46]:
# output_file('strava_chart_grid.html')

cds_bar = ColumnDataSource(data=daily_weekly_kms_df)

week_1_view = CDSView(source=cds_bar,
                       filters=[GroupFilter(column_name='week', group='week 1.0')])

# Create a figure with a datetime type x-axis
km_week_fig = figure(title='actual versus goals',
             plot_height=300, plot_width=600,
             x_axis_label='Day of the week 1.0', y_axis_label='KMs',
             x_range=x_axis,
             x_minor_ticks=2, y_range=(0, 50),
             toolbar_location=None)

# The daily words will be represented as vertical bars (columns)
km_week_fig.vbar(x='day_of_week', bottom=0, top='weekly_actual_cumulative', 
         color='blue', width=0.75, 
         legend='Actual', source=cds_bar, view=week_1_view)

# The cumulative sum will be a trend line
km_week_fig.line(x='day_of_week', y='weekly_goal_cumulative', 
         color='gray', line_width=1,
         legend='Goal', source=cds_bar, view=week_1_view)

# Put the legend in the upper left corner
km_week_fig.legend.location = 'top_left'

tooltips = [
            ('Cumulative Kilometers','@weekly_actual_cumulative'),
            ('Week number', '@week'),
           ]
# Add the HoverTool to the figure
km_week_fig.add_tools(HoverTool(tooltips=tooltips))


In [47]:
day_week_cds = ColumnDataSource(daily_weekly_kms_df)


weeks_comp_fig = figure(plot_height=300, plot_width=600,
                  title='Week 1 vs Week 2',
                  x_axis_label='Day of the week 1.0', y_axis_label='Kms',
                  x_range=x_axis,
                  toolbar_location=None, y_range=(0, 20))

week_1_view = CDSView(source=day_week_cds,
                       filters=[GroupFilter(column_name='week', group='week 1.0')])
week_2_view = CDSView(source=day_week_cds,
                       filters=[GroupFilter(column_name='week', group='week 2.0')])

weeks_comp_fig.line(x='day_of_week', y='kms', 
         color='blue', line_width=1,
         legend='Week 1',  source=day_week_cds, view=week_1_view)

weeks_comp_fig.line(x='day_of_week', y='kms', 
         color='red', line_width=1,
         legend='Week 2', source=day_week_cds, view=week_2_view)

# Move the legend to the upper left corner
weeks_comp_fig.legend.location = 'top_left'

tooltips = [
            (' Kilometers','@kms'),
           ]
# Add the HoverTool to the figure
weeks_comp_fig.add_tools(HoverTool(tooltips=tooltips))



In [48]:
all_week_gridplot = gridplot([[fig, km_fig], [weeks_comp_fig, km_week_fig], [stack_kms_goals, km_week_comp_fig]], 
                              toolbar_location='right')
# show(km_week_fig)
# show(km_fig)
# show(fig)
show(all_week_gridplot)

In [49]:
daily_weekly_kms_df.head(10)

,kms,cumulative_daily_actual_kms,daily_goal,cumulative_daily_goal,daily_kms_difference,daily_kms_cumulative_difference,week_number,week,weekly_actual_total,weekly_actual_cumulative,percentage_to_total_weekly,weekly_goal_total,weekly_goal_cumulative,percentage_to_total_weekly_goal,day_of_week,day_number,total_kms,diff_actual_vs_total,daily_kms_cumulative_difference_abs
run_date,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00+00:00,5.57,5.57,2.86,2.86,1.95,-34.47,1,week 1.0,62.89,5.57,0.088567,40.04,2.86,0.071429,None,1,1056.49,57.32,34.47
2018-01-02 00:00:00+00:00,5.77,11.34,2.86,5.71,2.02,-28.70,1,week 1.0,62.89,11.34,0.180315,40.04,5.72,0.142857,None,2,1056.49,51.55,28.70
2018-01-03 00:00:00+00:00,4.95,16.29,2.86,8.57,1.73,-23.75,1,week 1.0,62.89,16.29,0.259024,40.04,8.58,0.214286,None,3,1056.49,46.60,23.75
2018-01-04 00:00:00+00:00,0.75,17.04,2.86,11.43,0.26,-23.00,1,week 1.0,62.89,17.04,0.270949,40.04,11.44,0.285714,None,4,1056.49,45.85,23.00
2018-01-05 00:00:00+00:00,5.65,22.69,2.86,14.29,1.98,-17.35,1,week 1.0,62.89,22.69,0.360789,40.04,14.30,0.357143,None,5,1056.49,40.20,17.35
2018-01-06 00:00:00+00:00,6.02,28.71,2.86,17.14,2.11,-11.33,1,week 1.0,62.89,28.71,0.456511,40.04,17.16,0.428571,None,6,1056.49,34.18,11.33
2018-01-07 00:00:00+00:00,3.91,32.61,2.86,20.00,1.37,-7.43,1,week 1.0,62.89,32.62,0.518683,40.04,20.02,0.500000,None,7,1056.49,30.27,7.43
2018-12-31 00:00:00+00:00,0.00,1008.88,2.86,1042.86,0.00,968.84,1,week 1.0,62.89,32.62,0.518683,40.04,22.88,0.571429,None,8,1056.49,30.27,968.84
2019-01-01 00:00:00+00:00,8.06,1016.94,2.86,1045.71,2.82,976.90,1,week 1.0,62.89,40.68,0.646844,40.04,25.74,0.642857,None,9,1056.49,22.21,976.90


In [50]:
weekly_kms_df.to_csv('strava_weekly_data.csv')

In [51]:
weekly_kms_df

,kms,weekly_goal,cumulative_weekly_kms,cumulative_weekly_goal_kms,week_number,week
run_date,,,,,,
2018-01-01 00:00:00+00:00,5.57,2.86,5.57,2.86,1,week 1.0
2018-01-08 00:00:00+00:00,30.55,20.00,36.12,22.86,2,week 2.0
2018-01-15 00:00:00+00:00,38.93,20.00,75.05,42.86,3,week 3.0
2018-01-22 00:00:00+00:00,35.69,20.00,110.74,62.86,4,week 4.0
2018-01-29 00:00:00+00:00,21.71,20.00,132.45,82.86,5,week 5.0
2018-02-05 00:00:00+00:00,36.25,20.00,168.69,102.86,6,week 6.0
2018-02-12 00:00:00+00:00,22.83,20.00,191.52,122.86,7,week 7.0
2018-02-19 00:00:00+00:00,40.91,20.00,232.43,142.86,8,week 8.0
2018-02-26 00:00:00+00:00,29.39,20.00,261.82,162.86,9,week 9.0


In [52]:
daily_weekly_kms_df['annual_total'] = daily_weekly_kms_df['kms'].sum()

In [53]:
daily_weekly_kms_df.head()

,kms,cumulative_daily_actual_kms,daily_goal,cumulative_daily_goal,daily_kms_difference,daily_kms_cumulative_difference,week_number,week,weekly_actual_total,weekly_actual_cumulative,percentage_to_total_weekly,weekly_goal_total,weekly_goal_cumulative,percentage_to_total_weekly_goal,day_of_week,day_number,total_kms,diff_actual_vs_total,daily_kms_cumulative_difference_abs,annual_total
run_date,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:00:00+00:00,5.57,5.57,2.86,2.86,1.95,-34.47,1,week 1.0,62.89,5.57,0.088567,40.04,2.86,0.071429,None,1,1056.49,57.32,34.47,1056.49
2018-01-02 00:00:00+00:00,5.77,11.34,2.86,5.71,2.02,-28.70,1,week 1.0,62.89,11.34,0.180315,40.04,5.72,0.142857,None,2,1056.49,51.55,28.70,1056.49
2018-01-03 00:00:00+00:00,4.95,16.29,2.86,8.57,1.73,-23.75,1,week 1.0,62.89,16.29,0.259024,40.04,8.58,0.214286,None,3,1056.49,46.60,23.75,1056.49
2018-01-04 00:00:00+00:00,0.75,17.04,2.86,11.43,0.26,-23.00,1,week 1.0,62.89,17.04,0.270949,40.04,11.44,0.285714,None,4,1056.49,45.85,23.00,1056.49
2018-01-05 00:00:00+00:00,5.65,22.69,2.86,14.29,1.98,-17.35,1,week 1.0,62.89,22.69,0.360789,40.04,14.30,0.357143,None,5,1056.49,40.20,17.35,1056.49
